In [1]:
!pip install gensim
!pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 24.0/24.0 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 58.6/58.6 KB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 5.3 MB/s eta 0:00:00


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 50.5/50.5 KB 856.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for python-Levenshtein
Failed to build python-Levenshtein
  Running setup.py install for python-Levenshtein: started
  Running setup.py install for python-Levenshtein: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [28 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\Levenshtein
  copying Levenshtein\StringMatcher.py -> build\lib.win-amd64-3.8\Levenshtein
  copying Levenshtein\__init__.py -> build\lib.win-amd64-3.8\Levenshtein
  running egg_info
  writing python_Levenshtein.egg-info\PKG-INFO
  writing dependency_links to python_Levenshtein.egg-info\dependency_links.txt
  deleting python_Levenshtein.egg-info\entry_points.txt
  writing namespace_packages to python_Levenshtein.egg-info\namespace_packages.txt
  writing requirements to python_Levenshtein.egg-info\requires.txt
  writing top-level names to python_Levenshtein.egg-info\top_level.txt
  reading manifest file 'python_Levenshtein.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'CO

In [48]:
import numpy as np

In [2]:
import gensim
import pandas as pd

In [4]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())

True


In [8]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Reading and Exploring the Dataset
The dataset we are using here is a subset of Amazon reviews from the Cell Phones & Accessories category. The data is stored as a JSON file and can be read using pandas.

Link to the Dataset: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

In [11]:
df = pd.read_json("reviews_Cell_Phones_and_Accessories_5.json.gz", lines=True)
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
...,...,...,...,...,...,...,...,...,...
194434,A1YMNTFLNDYQ1F,B00LORXVUE,eyeused2loveher,"[0, 0]",Works great just like my original one. I reall...,5,This works just perfect!,1405900800,"07 21, 2014"
194435,A15TX8B2L8B20S,B00LORXVUE,Jon Davidson,"[0, 0]",Great product. Great packaging. High quality a...,5,Great replacement cable. Apple certified,1405900800,"07 21, 2014"
194436,A3JI7QRZO1QG8X,B00LORXVUE,Joyce M. Davidson,"[0, 0]","This is a great cable, just as good as the mor...",5,Real quality,1405900800,"07 21, 2014"
194437,A1NHB2VC68YQNM,B00LORXVUE,Nurse Farrugia,"[0, 0]",I really like it becasue it works well with my...,5,I really like it becasue it works well with my...,1405814400,"07 20, 2014"


In [12]:
df.shape

(194439, 9)

### Simple Preprocessing & Tokenization
The first thing to do for any data science task is to clean the data.
For NLP, we apply various processing like converting all the words to lower case, trimming spaces, removing punctuations. 
This is something we will do over here too.

Additionally, we can also remove stop words like 'and', 'or', 'is', 'the', 'a', 'an' and convert words to their root forms like 'running' to 'run'.

In [13]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)

In [14]:
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [15]:
review_text.loc[0]

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [16]:
df.reviewText.loc[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

### Training the Word2Vec Model

Train the model for reviews. Use a window of size 10 i.e. 10 words before the present word and 10 words ahead. A sentence with at least 2 words should only be considered, configure this using min_count parameter.

Workers define how many CPU threads to be used.

#### Initialize the model

In [102]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=8,
)


#### Build Vocabulary

In [106]:
model.build_vocab(review_text, progress_per=1000)

#### Train the Word2Vec Model

In [19]:
model.corpus_count

194439

In [107]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61851306, 79662910)

### Save the Model

Save the model so that it can be reused in other applications

In [108]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

### Finding Similar Words and Similarity between words
https://radimrehurek.com/gensim/models/word2vec.html

In [109]:
model.wv.most_similar("awesome")

[('amazing', 0.920524001121521),
 ('fantastic', 0.831725537776947),
 ('wonderful', 0.8087378740310669),
 ('awsome', 0.7773715257644653),
 ('fabulous', 0.7762017250061035),
 ('great', 0.7498964071273804),
 ('unbelievable', 0.7205672860145569),
 ('excellent', 0.7204397320747375),
 ('outstanding', 0.7102950215339661),
 ('terrific', 0.670698881149292)]

In [110]:
model.wv.similarity(w1="terrible", w2="awful")

0.82905644

In [111]:
model.wv.similarity(w1="great", w2="good")

0.7939214

In [112]:
model.wv.doesnt_match(['great',"terrible","awful"])

'great'

In [113]:
model.wv.get_vector('awesome')

array([-3.1357440e-01, -1.8846503e+00,  1.5905714e-01,  2.0867603e+00,
       -3.0972159e+00,  8.3842099e-02, -1.0050424e+00, -1.2464539e+00,
       -5.4026181e-01,  7.1587216e-04, -1.2459315e+00, -5.5784005e-01,
        7.0549560e-01, -1.7840985e+00,  1.0961343e+00, -5.6400055e-01,
       -1.7303877e-01,  5.3809530e-01, -3.0265839e+00, -1.8614974e+00,
       -1.2322328e+00, -3.5947065e+00, -1.0773315e+00, -1.0238154e+00,
       -8.9988804e-01,  2.6225123e-01, -9.0086871e-01, -9.0875876e-01,
        2.8117239e-01,  2.1094632e+00, -5.7892227e-01, -6.7939681e-01,
        2.1981995e+00,  4.8940521e-01, -1.2179322e+00,  8.4683454e-01,
       -4.4798568e-01,  2.9914875e+00, -5.0135674e+00,  6.9581968e-01,
       -2.1461177e-01,  4.0225630e+00,  1.8983421e+00, -1.7418339e+00,
       -7.7233618e-01,  4.4155955e+00,  3.7659585e+00,  4.2563543e+00,
       -1.0367882e+00,  2.3520267e+00, -1.6607495e-01,  3.5594565e-01,
        1.5833408e+00, -1.7389272e+00, -1.5581344e+00,  1.3654355e+00,
      

In [114]:
np.asarray(model.wv.vectors)

array([[ 1.7861643e+00, -1.0032858e-01,  6.6723728e-01, ...,
         1.8955928e+00,  1.2676752e-01,  3.9825004e-01],
       [ 6.5864062e-01,  9.2999764e-02, -1.5783429e+00, ...,
         5.1218289e-01,  7.7359855e-01, -1.1891801e+00],
       [ 1.1359007e+00,  2.5876389e+00, -2.0815918e+00, ...,
         3.4866457e+00,  4.6284409e+00, -1.3186277e+00],
       ...,
       [-5.2948087e-03,  8.7716035e-04,  1.3517974e-02, ...,
         5.8519938e-03, -1.0223205e-02, -3.0765412e-02],
       [ 2.9492456e-02,  2.9118137e-02, -7.8550857e-03, ...,
         4.0382851e-02, -1.9126693e-02, -7.6444961e-02],
       [-2.2509698e-02,  2.9320780e-02, -2.7761802e-02, ...,
         2.6892809e-02,  2.0536559e-03, -1.2866323e-02]], dtype=float32)

In [120]:
np.asarray(model.wv.index_to_key)[100:130]

array(['by', 'price', 'got', 'now', 'two', 'but_it', 'cable', 'you_can',
       'used', 'still', 'first', 'into', 'fits', 'using', 'it_was', 've',
       'cover', 'what', 'over', 'did', 'put', 'then', 'without', 'how',
       'protection', 'new', 'cases', 'buy', 'sound', 'way'], dtype='<U17')

In [117]:
model.wv.similar_by_vector(np.random.randint(100))

[('my_iphone', 0.7662007808685303),
 ('her_phone', 0.6663535833358765),
 ('phone', 0.640139102935791),
 ('it', 0.5837530493736267),
 ('it_still', 0.4780116677284241),
 ('pone', 0.4559711217880249),
 ('my_galaxy', 0.4496549367904663),
 ('my', 0.43737468123435974),
 ('floor', 0.42155447602272034),
 ('occasion', 0.4143906235694885)]

In [81]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

In [95]:
phrase_model = Phrases(review_text.loc[0:100], min_count=1, threshold=2, connector_words=ENGLISH_CONNECTOR_WORDS)

In [104]:
new_sentence = review_text
review_text=phrase_model[new_sentence]

In [88]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

In [124]:
mol=gensim.models.Word2Vec.load('word2vec-amazon-cell-accessories-reviews-short.model')

In [125]:
mol.wv.most_similar("awesome")

[('amazing', 0.920524001121521),
 ('fantastic', 0.831725537776947),
 ('wonderful', 0.8087378740310669),
 ('awsome', 0.7773715257644653),
 ('fabulous', 0.7762017250061035),
 ('great', 0.7498964071273804),
 ('unbelievable', 0.7205672860145569),
 ('excellent', 0.7204397320747375),
 ('outstanding', 0.7102950215339661),
 ('terrific', 0.670698881149292)]